# Wheatley

> Next-generation scheduling problem solver based on GNNs and Reinforcement Learning

The authors of Wheatley created a [script for training](jssp/train.py) and a [script for solving](jssp/solve.py) the problem. Documentation is in [USAGE.md](docs/USAGE.md) For this quick showcase, I trained a model on random 15x15 instances (15 jobs, 15 machines). To solve benchmark instances, I just need to import the function implemented by Wheatley's authors. The model takes quite some time to train, and therefore the model in this showcase is trained very badly due to time constraints. Makespans will be therefore incredibly bad.

In [1]:
# import Wheatley jssp solver
from jssp.solve import solve_instance
from jssp.utils.loaders import load_problem
from jssp.models.agent import Agent

# load trained agent
agent = Agent.load("saved_networks/agent.pkl")

# Benchmarks Wheatley can solve right now

Wheatley can solve only instances, which are the same size or smaller than those he has been trained on. Therefore I can 
I will now run Wheatley on all available JSSP benchmarks I currently have, which are 15x15 or smaller.

In [2]:
import os

def get_all_instances_in_taillard_specification():
    '''Lists all instances in Taillard specification'''
    matching_files = []
    root_dir = "../../../benchmarks/"
    target_string = "Taillard_specification"

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            if target_string in filepath:
                matching_files.append(filepath)

    return matching_files

In [3]:
for instance in sorted(get_all_instances_in_taillard_specification()):
    n_j, n_m, affectations, durations = load_problem(
        instance,
        taillard_offset=True,
        deterministic=True
    )

    if agent.env_specification.max_n_jobs < n_j:
        continue

    if agent.env_specification.max_n_machines < n_m:
        continue
        
    solution = solve_instance(
        agent, affectations, durations, True
    )
    print(f"Instance: {instance.split('/')[-1]}, jobs: {n_j}, machines: {n_m}, makespan: {solution.get_makespan()}")

/Users/marosbratko/Graph-neural-networks-and-deep-reinforcement-learning-in-job-scheduling/models/Wheatley/repo/jssp/env/observation.py:127: RuntimeWarning: invalid value encountered in cast
  "edge_index": edge_index.astype("int64"),


Instance: abz5.txt, jobs: 10, machines: 10, makespan: 3749.0
Instance: abz6.txt, jobs: 10, machines: 10, makespan: 2700.0
Instance: ft06.txt, jobs: 6, machines: 6, makespan: 80.0
Instance: ft10.txt, jobs: 10, machines: 10, makespan: 2768.0
Instance: la01.txt, jobs: 10, machines: 5, makespan: 1121.0
Instance: la02.txt, jobs: 10, machines: 5, makespan: 1171.0
Instance: la03.txt, jobs: 10, machines: 5, makespan: 1096.0
Instance: la04.txt, jobs: 10, machines: 5, makespan: 1296.0
Instance: la05.txt, jobs: 10, machines: 5, makespan: 839.0
Instance: la06.txt, jobs: 15, machines: 5, makespan: 1816.0
Instance: la07.txt, jobs: 15, machines: 5, makespan: 1801.0
Instance: la08.txt, jobs: 15, machines: 5, makespan: 2108.0
Instance: la09.txt, jobs: 15, machines: 5, makespan: 2001.0
Instance: la10.txt, jobs: 15, machines: 5, makespan: 2158.0
Instance: la16.txt, jobs: 10, machines: 10, makespan: 2673.0
Instance: la17.txt, jobs: 10, machines: 10, makespan: 2382.0
Instance: la18.txt, jobs: 10, machines: